In [2]:
library(tidyverse)
library(lme4)
library(ggplot2)

In [47]:
df_men <- read_csv("../data/temp_file_for_level_adj_men.csv")
df_women <- read_csv("../data/temp_file_for_level_adj_women.csv")


Parsed with column specification:
cols(
  Athlete_name = col_character(),
  Adjusted_score = col_double(),
  Season = col_double(),
  tier = col_double()
)

Parsed with column specification:
cols(
  Athlete_name = col_character(),
  Adjusted_score = col_double(),
  Season = col_double(),
  tier = col_double()
)



In [50]:
df_men['tier'] <- lapply(df_men['tier'], as.character)
df_men['Season'] <- lapply(df_men['Season'], as.character)


In [51]:
head(df_men)

Athlete_name,Adjusted_score,Season,tier
<chr>,<dbl>,<chr>,<chr>
COTA Jeremy - USA,107.983768,2010,1
DIGRAVIO David - USA,113.135247,2010,1
DISCOE Jimmy - USA,11.890073,2010,1
MARQUIS Philippe - CAN,106.418499,2010,1
ROCHON Cedric - CAN,106.396707,2010,1
DIGRAVIO David - USA,9.971327,2010,1


In [52]:
model_season <- lmer(Adjusted_score ~ tier + (1 | Season), data = df_men)
summary(model_season)


Linear mixed model fit by REML ['lmerMod']
Formula: Adjusted_score ~ tier + (1 | Season)
   Data: df_men

REML criterion at convergence: 5159.5

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.78822 -0.91112  0.01147  0.84977  2.72251 

Random effects:
 Groups   Name        Variance Std.Dev.
 Season   (Intercept)  187.6   13.70   
 Residual             2022.2   44.97   
Number of obs: 494, groups:  Season, 10

Fixed effects:
            Estimate Std. Error t value
(Intercept)   74.673      5.531  13.502
tier2        -21.468      4.559  -4.708
tier3        -27.862      6.806  -4.094

Correlation of Fixed Effects:
      (Intr) tier2 
tier2 -0.431       
tier3 -0.270  0.398

In [53]:
model_season_id <- lmer(Adjusted_score ~ tier + (1 | Season) + (1|Athlete_name), data = df_men)
summary(model_season_id)


Linear mixed model fit by REML ['lmerMod']
Formula: Adjusted_score ~ tier + (1 | Season) + (1 | Athlete_name)
   Data: df_men

REML criterion at convergence: 5159.4

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.75690 -0.90223  0.00634  0.85270  2.70110 

Random effects:
 Groups       Name        Variance Std.Dev.
 Athlete_name (Intercept)   13.71   3.703  
 Season       (Intercept)  176.33  13.279  
 Residual                 2011.14  44.846  
Number of obs: 494, groups:  Athlete_name, 65; Season, 10

Fixed effects:
            Estimate Std. Error t value
(Intercept)   74.649      5.466   13.66
tier2        -21.362      4.565   -4.68
tier3        -27.508      6.826   -4.03

Correlation of Fixed Effects:
      (Intr) tier2 
tier2 -0.439       
tier3 -0.280  0.401

In [54]:
anova(model_season, model_season_id)

refitting model(s) with ML (instead of REML)



,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model_season,5,5184.794,5205.807,-2587.397,5174.794,NA,NA,NA
model_season_id,6,5186.660,5211.875,-2587.330,5174.660,0.1348592,1,0.7134461


In [55]:
df_men['predicted_Scores'] <- predict(model_season, 
        newdata = df_men,
        allow.new.levels = TRUE)

In [69]:
df_men %>% group_by(Season) %>% summarize(avg_scores = mean(Adjusted_score))

Season,avg_scores
<chr>,<dbl>
2010,59.01445
2011,68.77741
2012,44.12599
2013,55.45862
2014,55.89153
2015,81.50650
2016,85.48224
2017,36.03873
2018,69.18387


In [99]:
df <- coef(model_season)$Season

In [103]:
df$tier2[1]

[1] -21.46763

In [104]:
df_all_men <- read_csv("../data/all_tiers_adjusted_scores_men.csv")
df_all_women <- read_csv("../data/all_tiers_adjusted_scores_women.csv")


Parsed with column specification:
cols(
  Athlete_Comp_ID = col_character(),
  Athlete_name = col_character(),
  Score_Metric = col_double(),
  Finish_Rank = col_double(),
  mean = col_double(),
  stddev = col_double(),
  Score_Multiplier = col_double(),
  Adjusted_score = col_double(),
  Season = col_double(),
  Season_Comp = col_character(),
  Round = col_character(),
  tier = col_double()
)

Parsed with column specification:
cols(
  Athlete_Comp_ID = col_character(),
  Athlete_name = col_character(),
  Score_Metric = col_double(),
  Finish_Rank = col_double(),
  mean = col_double(),
  stddev = col_double(),
  Score_Multiplier = col_double(),
  Adjusted_score = col_double(),
  Season = col_double(),
  Season_Comp = col_character(),
  Round = col_character(),
  tier = col_double()
)



In [108]:
df_all_men <- df_all_men %>% mutate(lvl_adj = case_when(tier == 1 ~ 0, tier == 2 ~ df$tier2[1], tier == 3 ~ df$tier3[1]))
df_all_women

Athlete_Comp_ID,Athlete_name,Score_Metric,Finish_Rank,mean,stddev,Score_Multiplier,Adjusted_score,Season,Season_Comp,Round,tier,lvl_adj
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
LAHTELA Janne - FIN - Olympics - 2002 - Final 1,LAHTELA Janne - FIN,93.23,1,87.31145,2.943088,1.9556744979,182.32753344,2002,2002-Olympics,Final 1,1,0
MAYER Travis - USA - Olympics - 2002 - Final 1,MAYER Travis - USA,91.97,2,85.42321,2.977849,1.9720863442,181.37278108,2002,2002-Olympics,Final 1,1,0
GAY Richard - FRA - Olympics - 2002 - Final 1,GAY Richard - FRA,89.70,3,84.07672,3.235895,1.9177509247,172.02225795,2002,2002-Olympics,Final 1,1,0
MOSELEY Jonny - USA - Olympics - 2002 - Final 1,MOSELEY Jonny - USA,89.27,4,82.89275,3.469604,1.9339426712,172.64306226,2002,2002-Olympics,Final 1,1,0
LUUSUA Tapio - FIN - Olympics - 2002 - Final 1,LUUSUA Tapio - FIN,88.90,5,80.99321,4.992387,1.8867540112,167.73243160,2002,2002-Olympics,Final 1,1,0
BELLAVANCE Scott - CAN - Olympics - 2002 - Final 1,BELLAVANCE Scott - CAN,88.50,6,76.65405,11.469266,1.6983227667,150.30156485,2002,2002-Olympics,Final 1,1,0
JOHNSON Ryan - CAN - Olympics - 2002 - Final 1,JOHNSON Ryan - CAN,88.50,7,75.75183,11.474051,1.7334503233,153.41035362,2002,2002-Olympics,Final 1,1,0
RONKAINEN Mikko - FIN - Olympics - 2002 - Final 1,RONKAINEN Mikko - FIN,88.30,8,74.64489,11.728169,1.7556978142,155.02811700,2002,2002-Olympics,Final 1,1,0
BLOOM Jeremy - USA - Olympics - 2002 - Final 1,BLOOM Jeremy - USA,87.30,9,73.21160,12.152023,1.7536847476,153.09667846,2002,2002-Olympics,Final 1,1,0


In [109]:
tail(df_all_men)

Athlete_Comp_ID,Athlete_name,Score_Metric,Finish_Rank,mean,stddev,Score_Multiplier,Adjusted_score,Season,Season_Comp,Round,tier
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
THUNDER Jericho - CAN - Canada Cup 2 - 2019 - Qualification 2,THUNDER Jericho - CAN,18.63,25,49.43726,20.36717,0.1303828,2.429031,2019,2019-Canada Cup 2,Qualification 2,3
GARDINER Reno - CAN - Canada Cup 2 - 2019 - Qualification 2,GARDINER Reno - CAN,17.62,26,48.91057,20.16156,0.1206644,2.126106,2019,2019-Canada Cup 2,Qualification 2,3
DUPRAY Adam - CAN - Canada Cup 2 - 2019 - Qualification 1,DUPRAY Adam - CAN,36.85,27,48.44902,20.47963,0.5711430,21.046621,2019,2019-Canada Cup 2,Qualification 1,3
PARSONS Nathan - CAN - Canada Cup 2 - 2019 - Qualification 1,PARSONS Nathan - CAN,33.17,28,47.37041,20.79277,0.4946388,16.407170,2019,2019-Canada Cup 2,Qualification 1,3
MASON Shawn - CAN - Canada Cup 2 - 2019 - Qualification 1,MASON Shawn - CAN,31.47,29,46.36180,21.07644,0.4798390,15.100535,2019,2019-Canada Cup 2,Qualification 1,3
PERNISIE Wylie - CAN - Canada Cup 2 - 2019 - Qualification 1,PERNISIE Wylie - CAN,29.87,30,45.12951,21.48840,0.4776249,14.266657,2019,2019-Canada Cup 2,Qualification 1,3
